In [1]:
#!/usr/bin/env python3

# Script to load the CLIMATE DATA .Rdata files and grid them and convert them to netcdf files per variable
# uses the .RData files retrieved from the RMI Oracle databse through the climate_grid_daily.txt script from Bert Van Schaeybroeck

# I. Vanderkelen, June 2024

import rpy2.robjects as robjects
from rpy2.robjects import pandas2ri
import xarray as xr
import pandas as pd
from datetime import date
import numpy as np
import rioxarray
import cartopy.crs as ccrs

# activate r to pandas convertor
pandas2ri.activate()


# user settings
variables = ["EVAPOTRANS_REF"]
#], "SUN_INT", "SUN_DURATION", "PRECIP_DURATION", "WIND_PEAK_SPEED", "PRECIP_1H_MAX", "EVAPOTRANS_REF", "TEMP_MAX","HUMIDITY_RELATIVE","TEMP_MIN", "TEMP_AVG", "WIND_SPEED", "PRESSURE", "SHORT_WAVE_FROM_SKY", "SUN_INT_HORIZ", "PRECIP_QUANTITY"]

# load climategrid meta data on variables and units
meta = pd.read_csv('CLIMATE_GRID_meta.csv', delimiter=";")

# load information on projection used 
proj_string = "+proj=lcc +lat_1=49.83333388888889 +lat_2=51.16666722222222 +lat_0=90 +lon_0=4.367486666666666 +x_0=150000.013 +y_0=5400088.438 +ellps=intl +units=m +no_defs"


start_yr = 1950
end_yr = 2023
dates = pd.date_range(start=f'{start_yr}-01-01', end=f'{end_yr}-12-31', freq='D')

data_dir = '/dodrio/scratch/projects/2022_200/project_output/RMIB-UGent/vsc31332_inne/CLIMATE_GRID/'


for variable in variables: 
	print('Converting '+ variable)

	filename = 'climate_atlas_'+str(variable)+'_CLIMATE_GRID_'+str(start_yr)+'_'+str(end_yr)+'.Rdata'

	# load the robject
	robjects.r['load'](data_dir+filename)

	# load the pixel lat and lon variable and use this to transpose to own defined grid
	df_coords =  pandas2ri.rpy2py(robjects.r['grid.pt.lst'])


	# define own grid  (in the original data, the delta's are not always the same, but the overall distance is similar)
	lons = df_coords['PIXEL_LON_CENTER']
	lon_min = lons.min()
	lon_max = lons.max()
	delta_lon = 0.0699 #°
	lon_grid = np.arange(lon_min,lon_max+delta_lon,delta_lon)

	lats = df_coords['PIXEL_LAT_CENTER']
	lat_min = lats.min()
	lat_max = lats.max()
	delta_lat = 0.045 #°
	lat_grid = np.arange(lat_min,lat_max+delta_lat,delta_lat)


	# Find the nearest index in the lons and lats grids and add this to coordinates dataframe
	def find_nearest(array, value):
		idx = (np.abs(array - value)).argmin()
		return idx

	df_coords['LON_INDEX'] = df_coords['PIXEL_LON_CENTER'].apply(lambda x: find_nearest(lon_grid, x))
	df_coords['LAT_INDEX'] = df_coords['PIXEL_LAT_CENTER'].apply(lambda x: find_nearest(lat_grid, x))


	# load the vector data
	data_raw = robjects.r['grid.arr']

	#reshape around time-axis
	ntime = data_raw.shape[1] * data_raw.shape[2] * data_raw.shape[3]
	data = data_raw.reshape(1360, ntime)

	# reshape around lat lon axis

	# create empty array to fill with gridded data
	grid_data = np.full(( ntime,  len(lat_grid),len(lon_grid)  ), np.nan)

	# Fill the grid data array
	for index, row in df_coords.iterrows():
		lat_idx = int(row['LAT_INDEX'])
		lon_idx = int(row['LON_INDEX'])
		pixel_id = int(row['PIXEL_ID'])
		
		grid_data[:, lat_idx, lon_idx ] = data[int(pixel_id) - 1, :]


	# time selection - the GRIDDATA is downloaded up to the last day 
	# only select full years here
	grid_data = grid_data[:len(dates),:,:]


	# get metadata from meta dataframe
	unit = meta.loc[meta['variable'] == variable, 'unit'].values[0] 
	long_name = meta.loc[meta['variable'] == variable, 'long_name'].values[0] 
	description = meta.loc[meta['variable'] == variable, 'description'].values[0] 

	# create data array
	da = xr.DataArray(
		data=grid_data,
		dims=["time",  "lat", "lon"],
		coords=dict(
			lon=lon_grid,
			lat=lat_grid,
			time=dates,
		),
		attrs=dict(
			long_name=long_name,
			description = description,
			units=unit,
		),
	)

	da['lat'].attrs = {'units':"degrees_north", 'long_name': "latitude"}
	da['lon'].attrs = {'units':"degrees_east", 'long_name': "longitude"}

	# convert to dataset and give dataset attributes
	ds = da.to_dataset(name=variable)
	d_attrs = {"creation_date": date.today().strftime("%d-%m-%Y"), "creators": "Ghilain N., Van Schaeybroeck B., Vanderkelen I.", 
	"contact": "inne.vanderkelen@meteo.be", "version": "1.1", "affiliation": "Royal Meteorological Institute of Belgium"}
	ds.attrs = d_attrs

	# export to netcdf
	filename_out = str(variable)+'_CLIMATE_GRID_'+str(start_yr)+'_'+str(end_yr)+'_daily.nc'
	ds.to_netcdf(data_dir + filename_out, encoding={'time':  {'dtype': 'int32'} })



Converting EVAPOTRANS_REF


In [2]:
robjects.r['load'](data_dir + filename)

# List all objects in the R environment
all_objects = robjects.r.ls()

# Print all objects
print("Objects in R environment after loading the file:")
for obj in all_objects:
    print(obj)

Objects in R environment after loading the file:
grid.arr
grid.pt.lst
grid.vec
munic.arr
munic.vec


In [1]:
#!/usr/bin/env python3

# Script to load the CLIMATE DATA .Rdata files and grid them and convert them to netcdf files per variable
# uses the .RData files retrieved from the RMI Oracle databse through the climate_grid_daily.txt script from Bert Van Schaeybroeck

# I. Vanderkelen, June 2024

import rpy2.robjects as robjects
from rpy2.robjects import pandas2ri
import xarray as xr
import rioxarray

import pandas as pd
from datetime import date
import numpy as np
import cartopy.crs as ccrs


# activate r to pandas convertor
pandas2ri.activate()


# user settings
variables = ["TEMP_MAX"]
#"EVAPOTRANS_REF", "SUN_INT", "SUN_DURATION", "PRECIP_DURATION", "WIND_PEAK_SPEED", "PRECIP_1H_MAX", "EVAPOTRANS_REF", "TEMP_MAX","HUMIDITY_RELATIVE","TEMP_MIN", "TEMP_AVG", "WIND_SPEED", "PRESSURE", "SHORT_WAVE_FROM_SKY", "SUN_INT_HORIZ", "PRECIP_QUANTITY"]


# INFO ON GRID
# load climategrid meta data on variables and units
meta = pd.read_csv('CLIMATE_GRID_meta.csv', delimiter=";")

# load information on projection used 
proj_string = "+proj=lcc +lat_1=49.83333388888889 +lat_2=51.16666722222222 +lat_0=90 +lon_0=4.367486666666666 +x_0=150000.013 +y_0=5400088.438 +ellps=intl +units=m +no_defs"
# load the pixel lat and lon variable and use this to transpose to own defined grid
df_coords_points = pd.read_csv('grid_5kmx5km.csv', header=1, delimiter=' ') # these are the lat lons and lambert coordinates for all pixels in CLIMATE_DATA


# load the full grid, creased based on the proj_string and following bounding points:
#NE_lon, NE_lat = 9.53269211610237, 53.4367017362904
#SW_lon, SW_lat = 0.163155782953472, 47.515819098539
df_full_grid = pd.read_csv('lambert_coordinates_full_climate_grid.csv') # made using R script Michel Journee
lambert_x_grid_raw = df_full_grid['x1'].unique()
lambert_y_grid_raw = df_full_grid['x2'].unique()

# cut this grid to boundingbox including gridcells from CLIMATE_GRID
lambert_x_grid_cutlow = lambert_x_grid_raw[lambert_x_grid_raw>=df_coords_points['LAMBERT_X'].min()]
lambert_x_grid = lambert_x_grid_cutlow[lambert_x_grid_cutlow<=  df_coords_points['LAMBERT_X'].max()]
lambert_y_grid_cutlow = lambert_y_grid_raw[lambert_y_grid_raw>=df_coords_points['LAMBERT_Y'].min()]
lambert_y_grid = lambert_y_grid_cutlow[lambert_y_grid_cutlow<=  df_coords_points['LAMBERT_Y'].max()]


# Find the nearest index in the lons and lats grids and add this to coordinates dataframe
def find_nearest(array, value):
	idx = (np.abs(array - value)).argmin()
	return idx

df_coords_points['LAMBERT_X_INDEX'] = df_coords_points['LAMBERT_X'].apply(lambda x: find_nearest(lambert_x_grid, x))
df_coords_points['LAMBERT_Y_INDEX'] = df_coords_points['LAMBERT_Y'].apply(lambda x: find_nearest(lambert_y_grid, x))


data_dir = '/dodrio/scratch/projects/2022_200/project_output/RMIB-UGent/vsc31332_inne/CLIMATE_GRID/'

for variable in variables: 
	print('Converting '+ variable)

	filename = 'climate_atlas_'+str(variable)+'_CLIMATE_GRID_1950_2023.Rdata'

	# load the robject
	robjects.r['load'](data_dir+filename)


	# load the time data and convert to dates
	time = robjects.r['time.lst']
	dates = pd.to_datetime(time)

	# load the vector data
	data = robjects.r['grid.vec']

	# create empty array to fill with gridded data - also for lat and lon
	grid_data = np.full(( len(dates),  len(lambert_y_grid),len(lambert_x_grid)  ), np.nan)

	lat_2d = np.full((len(lambert_y_grid),len(lambert_x_grid)  ), np.nan)
	lon_2d = np.full((len(lambert_y_grid),len(lambert_x_grid)  ), np.nan)

	# Fill the grid data array
	for index, row in df_coords_points.iterrows():
		lambert_x_idx = int(row['LAMBERT_Y_INDEX'])
		lambert_y_idx = int(row['LAMBERT_X_INDEX'])

		pixel_id = int(row['PIXEL_ID'])
		
		grid_data[:, lambert_x_idx, lambert_y_idx ] = data[int(pixel_id) - 1, :]

		lat_2d[ lambert_x_idx, lambert_y_idx ] = df_coords_points[df_coords_points['PIXEL_ID'] == pixel_id]["LAT"].values[0]
		lon_2d[ lambert_x_idx, lambert_y_idx ] = df_coords_points[df_coords_points['PIXEL_ID'] == pixel_id]["LON"].values[0]


	# get metadata from meta dataframe
	unit = meta.loc[meta['variable'] == variable, 'unit'].values[0] 
	long_name = meta.loc[meta['variable'] == variable, 'long_name'].values[0] 
	description = meta.loc[meta['variable'] == variable, 'description'].values[0] 

	# create data array
	da = xr.DataArray(
		data=grid_data,
		dims=["time", "y", "x"],
		coords=dict(
			y=lambert_y_grid,
			x=lambert_x_grid,
			time=dates,
		),
		attrs=dict(
			long_name=long_name,
			description = description,
			units=unit,
		),
	)


	da['x'].attrs = {'units':"E[east]: Easting (meters)", 'long_name': " x coordinate Lambert Conic Conformal (2SP)"}
	da['y'].attrs = {'units':"N[north]: Northing (meters)", 'long_name': "y coordinate Lambert Conic Conformal (2SP)"}

	# convert to dataset and give dataset attributes
	ds = da.to_dataset(name=variable)

	# add 2d lat and lon in lambert coordinates as well
	ds["lat"] = xr.DataArray(
		data=lat_2d,
		dims=["y", "x"],
		coords=dict(
			y=lambert_y_grid,
			x=lambert_x_grid,
		),
		attrs=dict(
			long_name="latitude",
			description = "WGS84 latitude, from values of CLIMATE_GRID, provided per grid point",
			units="degrees_north",
		),
	)
	
	# interpolatie to also have lat values outside of Belgium
	ds['lat'] = ds['lat'].interpolate_na(dim='x', method='linear', fill_value="extrapolate")

	ds["lon"] = xr.DataArray(
		data=lon_2d,
		dims=["y", "x"],
		coords=dict(
			y=lambert_y_grid,
			x=lambert_x_grid,
		),
		attrs=dict(
			long_name="longitude",
			description = "WGS84 longitude, from values of CLIMATE_GRID, provided per grid point",
			units="degrees_east",
		),
	)
	d_attrs = {"creation_date": date.today().strftime("%d-%m-%Y"),
	"creators": "Ghilain N., Van Schaeybroeck B., Vanderkelen I.", 
	"contact": "inne.vanderkelen@meteo.be",
	"version": "1.1", "affiliation": "Royal Meteorological Institute of Belgium", 
	"projection":proj_string}
	ds.attrs = d_attrs

	# interpolatie to also have lon values outside of Belgium
	ds['lon'] = ds['lon'].interpolate_na(dim='x', method='linear', fill_value="extrapolate")

	# also pass crs using rioxarray - passing the spatial_ref
	ds.rio.write_crs(ccrs.Projection(proj_string), inplace=True)

	# export to netcdf
	filename_out = str(variable)+'_CLIMATE_GRID_'+str(dates.year.min())+'_'+str(dates.year.max())+'_daily.nc'
	ds.to_netcdf(data_dir + filename_out, encoding={'time':  {'dtype': 'int32'} })



Converting TEMP_MAX


In [5]:
ds['lat'].min()

<xarray.DataArray 'lat' ()> Size: 8B
array(49.543)
Coordinates:
    spatial_ref  int64 8B 0

In [2]:
# regrid to lat lon grid

# Define the target grid (for example, a regular grid in lat/lon)
lat_new = np.linspace(ds['lat'].min(), ds['lat'].max(), 43)
lon_new = np.linspace(ds['lon'].min(), ds['lon'].max(), 57)
grid_out = xr.Dataset({'lat': (['lat'], lat_new), 'lon': (['lon'], lon_new)})



In [3]:
grid_out

<xarray.Dataset> Size: 800B
Dimensions:  (lat: 43, lon: 57)
Coordinates:
  * lat      (lat) float64 344B 49.54 49.59 49.64 49.68 ... 51.4 51.45 51.49
  * lon      (lon) float64 456B 2.474 2.545 2.616 2.688 ... 6.321 6.392 6.463
Data variables:
    *empty*

In [4]:
# define own grid
import xesmf as xe


# Create the regridder
regridder = xe.Regridder(ds, grid_out, 'conservative')

# Perform the regridding for TEMP_MAX variable
da_regridded = regridder(ds['TEMP_MAX'])
da_regridded

ModuleNotFoundError: No module named 'xesmf'

In [5]:
da_regridded.mean('time').plot()

NameError: name 'da_regridded' is not defined